# Geodatenanalyse 2


## Termin 10 - Modul 1

## Einführung in *Google Earth Engine* (*ee*)

Ca. 20-30 Minuten

## Inhalt

- GIS Datentypen
- Was ist Google Earth Engine?
- Interaktive Karten
- Dateneigenschaften
- Räumliche Dimensionen
- Zeitliche Dimensionen

## Geemap Installation

Für dieses Modul brauchen wir [geemap](https://geemap.org). Zur Installation bitte:

1 - *Anaconda Prompt* öffnen und folgendes eingeben:

2 - ``conda activate geo`` ENTER

3 - ``conda install mamba -c conda-forge`` ENTER

4 - ``mamba install geemap xarray_leaflet -c conda-forge`` ENTER

5 - ``jupyter nbextension enable --py --sys-prefix ipyleaflet`` ENTER

In [3]:
import ee
# initialize the connection to the server
ee.Initialize()
import geemap
import geemap.colormaps as cm
import datetime as dt
import pandas as pd

## GIS Datentypen

Quelle: [CVRD Canada](https://www.cvrd.ca/3239/GIS-Downloads)

<img width=500 style="float: left;" src="images\GIS_overview.png">

## Was ist *Google Earth Engine* (*ee*)?

Siehe Erklärung auf der [Google Webseite](https://earthengine.google.com/)

<img width=500 style="float: left;" src="images\gee_overview.png">

### Ein Blick in den Datenkatalog

- *ee* bietet unglaublich viele Möglichkeiten für die Geodatenanalyse

- Für verfügbare Datensätze, siehe den [Datenkatalog](https://developers.google.com/earth-engine/datasets/catalog)

### Der *Google Earth Engine* Explorer

Ein [Browser-basiertes Tool](https://explorer.earthengine.google.com/#workspace) zur Übersicht von Datensätzen

<img width=800 style="float: left;" src="images\ee_explorer.png">

### Datentypen in *Google ee*

- **ee.Features**, die geometrische Objekte mit einer Liste von Eigenschaften sind


- **ee.Images**, die wie Features sind, aber mehrere Bänder enthalten können


- **ee.Collections**, die Gruppen von Features oder Bildern sind
 - **ee.FeatureCollection** eine gebündelte Anzahl an *ee.Features*
 - **ee.ImageCollection** eine bliebige Anzahl an *ee.Images*

*Hinweis*: Die [Developer Guides](https://samapriya.github.io/gee-py/projects/collection_meta/) enthalten viele hilfreiche Informationen und Rezepte für die Datenanalyse.

### Wichtige Hinweise zum Umgang mit *Earth Engine*

- Die eigentlichen Daten liegen auf einem Google Server und verbrauchen sehr viel Speicherplatz

- Python bietet eine Schnittstelle in die Datenbank

- Python-Code innerhalb von *ee* funktioniert ähnlich wie SQL: Er führt eine Datenbankabfrage auf dem Server aus

- Oftmals ist es wichtig, dass man die gewünschten Daten vor dem Download geziehlt auswählt

- Ansonsten muss man lange auf die Anfrage warten oder man kriegt eine Warnung wegen Speicherbegrenzung

Ein sehr guter Überblick über die Syntax gibt es im [GEE Tutorials](https://tutorials.geemap.org)

## Ein Bild als Raster 

Quelle: [ArcGIS](https://desktop.arcgis.com/en/arcmap/10.3/manage-data/geodatabases/raster-basics.htm)

<img width=400 style="float: left;" src="images\image_raster.gif">

## Interaktive Karten

- Das *Earth Engine* *Advanced Programming Interface* (*API*) für Python lernt man am besten in Kombination mit interaktiven Karten

- Mit dem Python-Paket *geemap* lassen sich ganz einfach interaktive Karten erstellen

- Damit lassen sich Datensätze grafisch untersuchen, bearbeiten und exportieren


In [4]:
KA_map = geemap.Map(location=[49.014, 8.405], zoom_start=14)
KA_map

Map(center=[49.014, 8.405], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [5]:
KA_map = geemap.Map(location=[49.014, 8.405], zoom_start=14, basemap='HYBRID')
KA_map

Map(center=[49.014, 8.405], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

### *ee.Image* in die Karte laden

- Als Beispiel laden wir ein digitales Höhenmodell in die Karte

- Das Höhenmodell ist von der [NASA Shuttle Radar Topography Mission](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003)

- Die horizontale Auflösung der Pixel ist ungefär 1 Arcsekunde oder 30 m auf der Erdoberfläche

In [6]:
# load an image ...
image = ee.Image('USGS/SRTMGL1_003')

# visualise the image on a dynamic map
EU_map = geemap.Map(location=[49.014, 8.405], zoom_start=5)
vis_params = {'min': 0, 'max': 4000, 'palette': cm.palettes.dem}
EU_map.addLayer(image, vis_params, 'SRTM DEM', True, 0.7)

EU_map

Map(center=[49.014, 8.405], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

### *ee.Feature* in die Karte laden

- Als nächstes Beispiel laden wir eine Karte der Abflüsse

- Dieser Datensatz bietet Polylinien, die Flussnetzwerke darstellen, abgeleitet von und konsistent mit anderen HydroSHEDS-Datensätzen

- Die Daten stammen von der [HydroSHEDS Free Flowing Rivers Network v1](https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_v1_FreeFlowingRivers)

- Die Auflösung ist ungefär 15 Arcsekunden oder 500 m am Äquator

In [7]:
feature = ee.FeatureCollection('WWF/HydroSHEDS/v1/FreeFlowingRivers')

globe = geemap.Map(location=[49.014, 8.405], zoom_start=10)
vis_params = {'color': "024ACA", 'width': 1.0}
globe.addLayer(feature, vis_params, name='Rivers')
globe

Map(center=[49.014, 8.405], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

## Dateneigenschaften

Am wichtigsten sind die Metadaten eines Rasters:

In [8]:
# print the MetaData
metadata = image.getInfo()
metadata

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [1296001, 417601],
   'crs': 'EPSG:4326',
   'crs_transform': [0.0002777777777777778,
    0,
    -180.0001388888889,
    0,
    -0.0002777777777777778,
    60.00013888888889]}],
 'id': 'USGS/SRTMGL1_003',
 'version': 1641990767055141,
 'properties': {'system:visualization_0_min': '0.0',
  'type_name': 'Image',
  'keywords': ['dem',
   'elevation',
   'geophysical',
   'nasa',
   'srtm',
   'topography',
   'usgs'],
  'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png',
  'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="https://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JP

Diese sind als *dictionary* Objekt gespeichert und enthlten sämtliche Beschreibungen zum Datensatz.

In [9]:
metadata.keys()

dict_keys(['type', 'bands', 'id', 'version', 'properties'])

### Bildkanäle

- *ee.Images* können sogennante *bands* (auf deutsch Kanäle) enthalten

- Diese können aus unterschiedlichen Eigenschaften bestehen

- Beispiel: *Red Green Blue* (RGB) für das sichtbare Spektrum

Quelle: [Humboldt State University](https://gsp.humboldt.edu/OLM/Courses/GSP_216_Online/lesson3-1/bands.html)

<img width=500 style="float: left;" src="images\image_bands.jpg">

In [10]:
# load the image ...
landsat = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

In [11]:
# Get information about the bands as a list
bandNames = landsat.bandNames()
print('Band names: ' + str(bandNames.getInfo()))

Band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']


Bildkanäle können gezielt ausgewählt werden:

In [12]:
img_b3 = landsat.select('B3')
img_b3.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7661, 7801],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 460785, 0, -30, 4264215]}],
 'id': 'LANDSAT/LC08/C01/T1/LC08_044034_20140318',
 'version': 1497472259022000.0,
 'properties': {'RADIANCE_MULT_BAND_5': 0.006170900072902441,
  'RADIANCE_MULT_BAND_6': 0.001534600043669343,
  'RADIANCE_MULT_BAND_3': 0.011958000250160694,
  'RADIANCE_MULT_BAND_4': 0.010084000416100025,
  'RADIANCE_MULT_BAND_1': 0.012672999873757362,
  'RADIANCE_MULT_BAND_2': 0.012977000325918198,
  'K2_CONSTANT_BAND_11': 1201.1441650390625,
  'K2_CONSTANT_BAND_10': 1321.078857421875,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-121.3637119499993, 36.41016684133052],
    [-121.35905784815819, 36.42528989660049],
    [-121.2315833015866, 36.840374852891664],
    [-121.09978718573184, 37.26438246506325],
    [-121.00571062336425, 37.564795

### Weitere Dateneigenschaften

In [13]:
# Get projection information from band B1
b1proj = landsat.select('B1').projection()
print('Band B1 projection: ' + str(b1proj.getInfo()))

Band B1 projection: {'type': 'Projection', 'crs': 'EPSG:32610', 'transform': [30, 0, 460785, 0, -30, 4264215]}


In [14]:
# Get scale (in meters) information from band 1.
elvscale = landsat.select('B1').projection().nominalScale()
print('Band B1 scale: ' + str(elvscale.getInfo()))

Band B1 scale: 30


In [15]:
# Get a list of all metadata properties.
properties = landsat.propertyNames()
print('Metadata properties: ' + str(properties.getInfo()))

Metadata properties: ['RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'system:id', 'K2_CONSTANT_BAND_11', 'K2_CONSTANT_BAND_10', 'system:footprint', 'REFLECTIVE_SAMPLES', 'SUN_AZIMUTH', 'CPF_NAME', 'DATE_ACQUIRED', 'ELLIPSOID', 'google:registration_offset_x', 'google:registration_offset_y', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_9', 'TARGET_WRS_ROW', 'RADIANCE_MULT_BAND_7', 'RADIANCE_MULT_BAND_8', 'IMAGE_QUALITY_TIRS', 'TRUNCATION_OLI', 'CLOUD_COVER', 'GEOMETRIC_RMSE_VERIFY', 'COLLECTION_CATEGORY', 'GRID_CELL_SIZE_REFLECTIVE', 'CLOUD_COVER_LAND', 'GEOMETRIC_RMSE_MODEL', 'COLLECTION_NUMBER', 'IMAGE_QUALITY_OLI', 'LANDSAT_SCENE_ID', 'WRS_PATH', 'google:registration_count', 'PANCHROMATIC_SAMPLES', 'PANCHROMATIC_LINES', 'GEOMETRIC_RMSE_MODEL_Y', 'REFLECTIVE_LINES', 'TIRS_STRAY_LIGHT_CORRECTION_SOURCE', 'GEOMETRIC_RMSE_MODEL_X', 'system:asset_size', 'syst

In [16]:
# Get a specific metadata property.
cloudiness = landsat.get('CLOUD_COVER')
print('CLOUD_COVER: '+str(cloudiness.getInfo()))

CLOUD_COVER: 0.05999999865889549


In [17]:
# Get the timestamp and convert it to a date.
date = landsat.get('system:time_start')
print('Time epoch: ' + str(date.getInfo()) + ' milliseconds (since 1970)')
dt_iso = dt.datetime.fromtimestamp(date.getInfo() // 1000)
print('Timestamp: ' + str(dt_iso))

Time epoch: 1395168392050 milliseconds (since 1970)
Timestamp: 2014-03-19 05:46:32


## Die räumliche Dimension

- Viele Datensätze haben eine vorgegebene regionale oder globale Abdeckung

- Oftmals möchte man nur Daten von bestimmte Regionen analysieren

- Für die Begrenzung gibt es eingebaute Funktionalität


In [18]:
# load the SRTM image
image = ee.Image('USGS/SRTMGL1_003')

# Create a circle by drawing a 20000 meter buffer around a point.
roi = ee.Geometry.Rectangle([8.1, 48.9, 8.7, 49.1])

# Clip the dataset ...
image_clipped = image.clip(roi)

In [19]:
# visualise the clipped dataset
globe = geemap.Map(location=[49.014, 8.405], zoom=10)
vis_params = {'min': 100, 'max': 400, 'palette': cm.palettes.dem}
globe.addLayer(image_clipped, vis_params, 'SRTM DEM', True, 0.9)

KA = ee.Geometry.Point([8.407, 49.015])
globe.addLayer(KA)

globe

Map(center=[49.014, 8.405], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [20]:
# Create a circle by drawing a 10000 meter buffer around a point.
roi = ee.Geometry.Point([8.405,49.014]).buffer(100000)

# clip the dataset
image_clipped = image.clip(roi)

# visualise the clipped dataset
globe = geemap.Map(location=[49.014, 8.405], zoom=8)
vis_params = {'min': 100, 'max': 1000, 'palette': cm.palettes.dem}
globe.addLayer(image_clipped, vis_params, 'SRTM DEM', True, 0.9)

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']
globe.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="DEM")

globe

Map(center=[49.014, 8.405], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

## *ee.ImageCollections* als Rasterserien

- Für viele Datensätze gibt es Serien, d.h. eine zeitliche Auflösung von Daten

- Diese werden immer als *ee.ImageCollections* gespeichert

- *ee.ImageCollections* bestehen aus einer Aneinanderreigung von *ee.Image*-Objekten

- Einzelne Zeitelemente können anhand von Metadaten extrahiert werden

### Inspektion der Metadaten

Der erste Schritt ist immer ein Blick auf die Metadaten eines einzelnen Rasters:

In [21]:
# load the NOAA/CDR/SST ImageCollection
collection = ee.ImageCollection('NOAA/CDR/SST_WHOI/V2')

# extract one image from the collection
image = ee.Image(collection.first())

# print the MetaData
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'sea_surface_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 720],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'fill_missing_qc',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [1440, 720],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}],
 'version': 1516960293924856,
 'id': 'NOAA/CDR/SST_WHOI/V2/19880101T00',
 'properties': {'system:time_start': 567993600000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 568004400000,
  'system:asset_size': 2048775,
  'system:index': '19880101T00'}}

In [22]:
# Get the number of images
count = collection.size().getInfo()
print('Number of images: ', str(count))

Number of images:  98136


In [23]:
# Get the date range of images in the collection
dates = collection.aggregate_array("system:time_start")

# convert from milliseconds to datetime format
datelist = pd.to_datetime(dates.getInfo(), unit='ms')

print('Start time: ' + str(datelist[0]))
print('End time: ' + str(datelist[-1]))

Start time: 1988-01-01 00:00:00
End time: 2021-08-31 21:00:00


### Extraktion von Elementen aus Kollektionen

Einzelne Daten können aus einer *ee.ImageCollection* extrahiert werden. Dieser Prozess wird als *filtern* bezeichnet.

Zuerst müssen wir die Informationen zusammenstellen. Hier zum Beispiel ein *index*:

In [24]:
# aggregate all indices in one array
img_ids = collection.aggregate_array("system:index")
print(img_ids.getInfo()[:10])

['19880101T00', '19880101T03', '19880101T06', '19880101T09', '19880101T12', '19880101T15', '19880101T18', '19880101T21', '19880102T00', '19880102T03']


Für die Extraktion von einzelnen Rastern ist *filterMetadata()* sehr praktisch. 

Achtung: Hier sollte man immer noch *first()* verwenden, um ein *ee.Image*-Objekt zu erhalten:

In [25]:
# extract a desired image from the collection
image = ee.Image(collection.filterMetadata('system:index','equals','19880213T12') \
                 .first())

# select a band ...
image_sst = image.select('sea_surface_temperature')
image_sst.getInfo()

{'type': 'Image',
 'bands': [{'id': 'sea_surface_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 720],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}],
 'version': 1516938555927735,
 'id': 'NOAA/CDR/SST_WHOI/V2/19880213T12',
 'properties': {'system:time_start': 571752000000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 571762800000,
  'system:asset_size': 2226788,
  'system:index': '19880213T12'}}

In [26]:
# visualise the image in a dynamic map
globe = geemap.Map(zoom=1)
vis_params = {'min': -5, 'max': 35, 'palette': cm.palettes.jet}
globe.addLayer(image_sst, vis_params, 'SST', True, 0.7)
globe

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Auch Datum und Zeit können zum Filtern verwendet werden. 

Schauen wir uns zuerst die verfügbaren Zeitdaten an:

In [27]:
# extract the date and time of images
dates = collection.aggregate_array("system:time_start")

# convert to a human readable format
datelist = pd.to_datetime(dates.getInfo(), unit='ms')
datelist

DatetimeIndex(['1988-01-01 00:00:00', '1988-01-01 03:00:00',
               '1988-01-01 06:00:00', '1988-01-01 09:00:00',
               '1988-01-01 12:00:00', '1988-01-01 15:00:00',
               '1988-01-01 18:00:00', '1988-01-01 21:00:00',
               '1988-01-02 00:00:00', '1988-01-02 03:00:00',
               ...
               '2021-08-30 18:00:00', '2021-08-30 21:00:00',
               '2021-08-31 00:00:00', '2021-08-31 03:00:00',
               '2021-08-31 06:00:00', '2021-08-31 09:00:00',
               '2021-08-31 12:00:00', '2021-08-31 15:00:00',
               '2021-08-31 18:00:00', '2021-08-31 21:00:00'],
              dtype='datetime64[ns]', length=98136, freq=None)

Für das Filtern bitte immer das ISO-8601 Format verwenden:

In [28]:
# extract image based on date and time
dated_image = ee.Image(collection.filterDate('2021-01-30T18:00:00').first())

# select the desired image band
dated_image = dated_image.select('sea_surface_temperature')

In [29]:
# visualise the image in a dynamic map
globe = geemap.Map(zoom=1)
vis_params = {'min': -1, 'max': 35, 'palette': cm.palettes.jet}
globe.addLayer(dated_image, vis_params, 'SST', True, 0.7)

globe

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Weitere Funktionalität

Dieser Überblick war nur ein Anfang der verfügbaren unktionalität von *ee* in Python.

Die [Developer Guides](https://samapriya.github.io/gee-py/projects/collection_meta/) enthalten viele hilfreiche Informationen und Rezepte für die Datenanalyse.

## ENDE